In [1]:
import sympy as sp
from sympy import cos as c, sin as s, pi, Function as f, Matrix as M


# Define the independent variable
x, y, t, l1, l2, m1, m2 = sp.symbols('x y t l1 l2 m1 m2')
q1, q2 = f('q1')(t), f('q2')(t)
c1, s1, c2, s2, c12, s12 = c(q1), s(q1), c(q2), s(q2), c(q1 + q2), s(q1 + q2)
t_diff = lambda expr: sp.diff(expr, t) if t in expr.free_symbols else expr

# Define the forward kinematics
R1 = M([[c1, -s1, 0], [s1, c1, 0], [0, 0, 1]])
T1 = M([[1, 0, l1], [0, 1, 0], [0, 0, 1]])
R2 = M([[c2, -s2, 0], [s2, c2, 0], [0, 0, 1]])
T2 = M([[1, 0, l2], [0, 1, 0], [0, 0, 1]])
Q01 = R1 * T1
Q12 = R2 * T2
Q02 = Q01 * Q12
Q02 = Q02.applyfunc(sp.simplify)
Ph = M([[x], [0], [1]])


Energia Cinetica Link 1

In [2]:
P = Q01*Ph
P = M([[P[0, 0]], [P[1, 0]]])
Pd = P.applyfunc(t_diff)
Pd2 = Pd.T * Pd
Pd2 = Pd2.applyfunc(sp.simplify)
Pd2

Matrix([[(l1 + x)**2*Derivative(q1(t), t)**2]])

In [3]:
rho1 = m1 / l1
tau1 = rho1*Pd2
T_L1 = sp.integrate(tau1, (x, -l1, 0))*1/2
T_L1

Matrix([[l1**2*m1*Derivative(q1(t), t)**2/6]])

In [4]:
T_M1 = m1*Pd2.subs(x, 0)*1/2
T_M1

Matrix([[l1**2*m1*Derivative(q1(t), t)**2/2]])

In [5]:
T1 = T_L1 + T_M1
T1

Matrix([[2*l1**2*m1*Derivative(q1(t), t)**2/3]])

Energia Cinetica Link2

In [6]:
P = Q02*Ph
P = M([[P[0, 0]], [P[1, 0]]])
Pd = P.applyfunc(t_diff)
Pd2 = Pd.T * Pd
Pd2 = Pd2.applyfunc(sp.simplify)
Pd2

Matrix([[(l1*sin(q1(t))*Derivative(q1(t), t) + l2*(Derivative(q1(t), t) + Derivative(q2(t), t))*sin(q1(t) + q2(t)) + x*(Derivative(q1(t), t) + Derivative(q2(t), t))*sin(q1(t) + q2(t)))**2 + (l1*cos(q1(t))*Derivative(q1(t), t) + l2*(Derivative(q1(t), t) + Derivative(q2(t), t))*cos(q1(t) + q2(t)) + x*(Derivative(q1(t), t) + Derivative(q2(t), t))*cos(q1(t) + q2(t)))**2]])

In [7]:
rho2 = m2 / l2
tau2 = rho2*Pd2
T_L2 = sp.integrate(tau2, (x, -l2, 0))*1/2
T_L2 = T_L2.applyfunc(sp.expand).applyfunc(sp.simplify)
T_L2

Matrix([[m2*(3*l1**2*Derivative(q1(t), t)**2 + 3*l1*l2*cos(q2(t))*Derivative(q1(t), t)**2 + 3*l1*l2*cos(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + l2**2*Derivative(q1(t), t)**2 + 2*l2**2*Derivative(q1(t), t)*Derivative(q2(t), t) + l2**2*Derivative(q2(t), t)**2)/6]])

In [8]:
T_M2 = m2*Pd2.subs(x, 0).applyfunc(sp.expand).applyfunc(sp.simplify)*1/2
T_M2

Matrix([[m2*(l1**2*Derivative(q1(t), t)**2 + 2*l1*l2*cos(q2(t))*Derivative(q1(t), t)**2 + 2*l1*l2*cos(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + l2**2*Derivative(q1(t), t)**2 + 2*l2**2*Derivative(q1(t), t)*Derivative(q2(t), t) + l2**2*Derivative(q2(t), t)**2)/2]])

In [9]:
T2 = (T_L2 + T_M2).applyfunc(sp.expand).applyfunc(sp.simplify)
T2

Matrix([[m2*(6*l1**2*Derivative(q1(t), t)**2 + 9*l1*l2*cos(q2(t))*Derivative(q1(t), t)**2 + 9*l1*l2*cos(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + 4*l2**2*Derivative(q1(t), t)**2 + 8*l2**2*Derivative(q1(t), t)*Derivative(q2(t), t) + 4*l2**2*Derivative(q2(t), t)**2)/6]])

In [10]:
T = (T1 + T2).applyfunc(sp.expand).applyfunc(sp.simplify)
T

Matrix([[2*l1**2*m1*Derivative(q1(t), t)**2/3 + l1**2*m2*Derivative(q1(t), t)**2 + 3*l1*l2*m2*cos(q2(t))*Derivative(q1(t), t)**2/2 + 3*l1*l2*m2*cos(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t)/2 + 2*l2**2*m2*Derivative(q1(t), t)**2/3 + 4*l2**2*m2*Derivative(q1(t), t)*Derivative(q2(t), t)/3 + 2*l2**2*m2*Derivative(q2(t), t)**2/3]])

In [11]:
def collect_with_respect_to_vars(eq, vars):
    assert isinstance(vars, list)
    eq = eq.expand()
    if len(vars) == 0:
        return {1: eq}

    var_map = eq.collect(vars[0], evaluate=False)
    final_var_map = {}
    for var_power in var_map:
        sub_expression = var_map[var_power]
        sub_var_map = collect_with_respect_to_vars(sub_expression, vars[1:])
        for sub_var_power in sub_var_map:
            final_var_map[var_power*sub_var_power] = sub_var_map[sub_var_power]
    return final_var_map

Now derivate the EL equations

In [12]:
# Auxiliary definitions for the derivatives
dq1 = q1.diff(t)
dq2 = q2.diff(t)
tau1, tau2 = sp.symbols('tau1 tau2')
# Define the Lagrangian
L = T   # assuming no potential energy and no external forces
dLddq1 = L.diff(dq1)
dLdq1 = L.diff(q1)
dLddq2 = L.diff(dq2)
dLdq2 = L.diff(q2)
# Define the Euler-Lagrange equations
eq1 = (dLddq1.diff(t) - dLdq1)[0]
eq2 = (dLddq2.diff(t) - dLdq2)[0]
# Solve the Euler-Lagrange equations
eq1 = eq1.expand().simplify()
eq2 = eq2.expand().simplify()
Ax = M([eq1, eq2])
Y = M([tau1, tau2])

In [50]:
# Solve for the accelerations
S = sp.Eq(Ax, Y)
sol = sp.solve(S, [q1.diff(t, t), q2.diff(t, t)])

{Derivative(q1(t), (t, 2)): 81*l1**2*l2*m2*sin(q2(t))*cos(q2(t))*Derivative(q1(t), t)**2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2) + 72*l1*l2**2*m2*sin(q2(t))*Derivative(q1(t), t)**2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2) + 144*l1*l2**2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t)/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2) + 72*l1*l2**2*m2*sin(q2(t))*Derivative(q2(t), t)**2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2) - 54*l1*tau2*cos(q2(t))/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2) + 48*l2*tau1/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2) - 48*l2*tau2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*cos(q2(t))**2 + 96*l1**2*l2*m2),
 Derivative(q2(t), (t, 2)): -72*l1**3*l2*m1*m2*sin(q2(t))*Derivative(q1(t), t)**2/(64*l1**2*l2**2*m1*m2 - 81*l1**2*l2**2*m2**2*cos(q2(t))**2 + 96*l1**2*l2**2*m2**2) - 108*l1**3*l2*m2**2*sin(q2(t))*Derivative(q1(t), t)**2/(64*l1**2

In [54]:
p = 81*l1**2*l2*m2*s2*c(q2)*dq1**2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2) + 72*l1*l2**2*m2*s2*dq1**2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2) + 144*l1*l2**2*m2*s2*dq1*dq2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2) + 72*l1*l2**2*m2*s2*dq2**2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2) - 54*l1*tau2*c2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2) + 48*l2*tau1/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2) - 48*l2*tau2/(64*l1**2*l2*m1 - 81*l1**2*l2*m2*c2**2 + 96*l1**2*l2*m2)

In [57]:
p.simplify()

3*(27*l1**2*l2*m2*sin(2*q2(t))*Derivative(q1(t), t)**2 + 48*l1*l2**2*m2*sin(q2(t))*Derivative(q1(t), t)**2 + 96*l1*l2**2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + 48*l1*l2**2*m2*sin(q2(t))*Derivative(q2(t), t)**2 - 36*l1*tau2*cos(q2(t)) + 32*l2*tau1 - 32*l2*tau2)/(l1**2*l2*(128*m1 - 81*m2*cos(2*q2(t)) + 111*m2))

In [36]:
eq1.collect([dq1.diff(t)], evaluate=False)

{Derivative(q1(t), (t, 2)): 4*l1**2*m1/3 + 2*l1**2*m2 + 3*l1*l2*m2*cos(q2(t)) + 4*l2**2*m2/3,
 1: -3*l1*l2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) - 3*l1*l2*m2*sin(q2(t))*Derivative(q2(t), t)**2/2 + 3*l1*l2*m2*cos(q2(t))*Derivative(q2(t), (t, 2))/2 + 4*l2**2*m2*Derivative(q2(t), (t, 2))/3 - tau1}

In [40]:
4*l1**2*m1/3 + 2*l1**2*m2 + 3*l1*l2*m2*c(q2) + 4*l2**2*m2/3

4*l1**2*m1/3 + 2*l1**2*m2 + 3*l1*l2*m2*cos(q2(t)) + 4*l2**2*m2/3

In [41]:
-3*l1*l2*m2*s(q2)*dq1*dq2 - 3*l1*l2*m2*s(q2)*dq2**2/2 + 3*l1*l2*m2*c(q2)*dq2.diff(t)/2 + 4*l2**2*m2*dq2.diff(t)/3 - tau1

-3*l1*l2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) - 3*l1*l2*m2*sin(q2(t))*Derivative(q2(t), t)**2/2 + 3*l1*l2*m2*cos(q2(t))*Derivative(q2(t), (t, 2))/2 + 4*l2**2*m2*Derivative(q2(t), (t, 2))/3 - tau1

In [14]:
L1, L2, g = sp.symbols('L1 L2 g')

$$\frac{d}{dt}\frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} = \tau_i$$
$$M(q)\ddot{q} + C(q,\dot{q})\dot{q} + G(q) = \tau$$

In [19]:
m_t = M([[m1*L1**2+m2*(L1**2+2*L1*L2*c2+L2**2), m2*(L1*L2*c2+L2**2)],
       [m2*(L1*L2*c2+L2**2), m2*L2**2]])
c_t_td = M([[-m2*L1*L2*s2*(2*dq1*dq2*dq2**2)],
       [m2*L1*L2*s2*dq1**2]])
g_t = M([[(m1+m2)*g*L1*c(q1)+m2*g*L2*c(q1+q2)],
       [m2*g*L2*c(q1+q2)]])

In [22]:
m_t_inv = m_t.inv()
m_t_inv

Matrix([
[                             1/(L1**2*m1 - L1**2*m2*cos(q2(t))**2 + L1**2*m2),                                                      (-L1*cos(q2(t)) - L2)/(L1**2*L2*m1 - L1**2*L2*m2*cos(q2(t))**2 + L1**2*L2*m2)],
[(-L1*cos(q2(t)) - L2)/(L1**2*L2*m1 - L1**2*L2*m2*cos(q2(t))**2 + L1**2*L2*m2), (L1**2*m1 + L1**2*m2 + 2*L1*L2*m2*cos(q2(t)) + L2**2*m2)/(L1**2*L2**2*m1*m2 - L1**2*L2**2*m2**2*cos(q2(t))**2 + L1**2*L2**2*m2**2)]])

$$\ddot{q} = M^{-1}(q)(\tau - C(q,\dot{q})\dot{q} - G(q))$$

In [26]:
eq = (m_t_inv*(M([[tau1],[tau2]])-c_t_td-g_t)).applyfunc(sp.simplify)
eq

Matrix([
[                                                                   (L2*(2*L1*L2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t)**3 - L1*g*(m1 + m2)*cos(q1(t)) - L2*g*m2*cos(q1(t) + q2(t)) + tau1) + (L1*cos(q2(t)) + L2)*(L1*L2*m2*sin(q2(t))*Derivative(q1(t), t)**2 + L2*g*m2*cos(q1(t) + q2(t)) - tau2))/(L1**2*L2*(m1 + m2*sin(q2(t))**2))],
[(-L2*m2*(L1*cos(q2(t)) + L2)*(2*L1*L2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t)**3 - L1*g*(m1 + m2)*cos(q1(t)) - L2*g*m2*cos(q1(t) + q2(t)) + tau1) - (L1*L2*m2*sin(q2(t))*Derivative(q1(t), t)**2 + L2*g*m2*cos(q1(t) + q2(t)) - tau2)*(L1**2*m1 + L1**2*m2 + 2*L1*L2*m2*cos(q2(t)) + L2**2*m2))/(L1**2*L2**2*m2*(m1 + m2*sin(q2(t))**2))]])

In [42]:
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
x = M([[x3],
       [x4],
       [eq[0,0].subs({q1:x1, q2:x2, dq1:x3, dq2:x4})],
       [eq[1,0].subs({q1:x1, q2:x2, dq1:x3, dq2:x4})]])
x

Matrix([
[                                                                                                                                                                                                                                                                  x3],
[                                                                                                                                                                                                                                                                  x4],
[                                                                (L2*(2*L1*L2*m2*x3*x4**3*sin(x2) - L1*g*(m1 + m2)*cos(x1) - L2*g*m2*cos(x1 + x2) + tau1) + (L1*cos(x2) + L2)*(L1*L2*m2*x3**2*sin(x2) + L2*g*m2*cos(x1 + x2) - tau2))/(L1**2*L2*(m1 + m2*sin(x2)**2))],
[(-L2*m2*(L1*cos(x2) + L2)*(2*L1*L2*m2*x3*x4**3*sin(x2) - L1*g*(m1 + m2)*cos(x1) - L2*g*m2*cos(x1 + x2) + tau1) - (L1*L2*m2*x3**2*sin(x2) + L2*g*m2*cos(x1 + x2) - tau2)*(L1**2*m1 + L1**2*m2 + 2*L1*L2

In [44]:
print(x[3,0])

(-L2*m2*(L1*cos(x2) + L2)*(2*L1*L2*m2*x3*x4**3*sin(x2) - L1*g*(m1 + m2)*cos(x1) - L2*g*m2*cos(x1 + x2) + tau1) - (L1*L2*m2*x3**2*sin(x2) + L2*g*m2*cos(x1 + x2) - tau2)*(L1**2*m1 + L1**2*m2 + 2*L1*L2*m2*cos(x2) + L2**2*m2))/(L1**2*L2**2*m2*(m1 + m2*sin(x2)**2))


(L2*(2*L1*L2*m2*x3*x4**3*sin(x2) - L1*g*(m1 + m2)*cos(x1) - L2*g*m2*cos(x1 + x2) + tau1) + (L1*cos(x2) + L2)*(L1*L2*m2*x3**2*sin(x2) + L2*g*m2*cos(x1 + x2) - tau2))/(L1**2*L2*(m1 + m2*sin(x2)**2))


(-L2*m2*(L1*cos(x2) + L2)*(2*L1*L2*m2*x3*x4**3*sin(x2) - L1*g*(m1 + m2)*cos(x1) - L2*g*m2*cos(x1 + x2) + tau1) - (L1*L2*m2*x3**2*sin(x2) + L2*g*m2*cos(x1 + x2) - tau2)*(L1**2*m1 + L1**2*m2 + 2*L1*L2*m2*cos(x2) + L2**2*m2))/(L1**2*L2**2*m2*(m1 + m2*sin(x2)**2))